# ДЗ1. Предобработка текстовых данных
Даша Максимова, БКЛ151

## Из материалов семинара

Смотрим на анализ тональности в твитах (соревнование Диалога прошлых лет).

In [1]:
import pandas as pd
import numpy as np

pd.set_option('max_colwidth', 1000)

### Данные

In [2]:
train_data = pd.read_csv("data/sentiment_twitter/train_sentiment_ttk.tsv", sep="\t")
test_data = pd.read_csv("data/sentiment_twitter/test_sentiment_ttk.tsv", sep="\t")

In [3]:
train_data.head()

,label,text
0,0,"@mkomov Максим, Вашем письмо мы получили. Наши сотрудники свяжутся с Вами завтра и направят запрос инженерам для проверки. #билайн"
1,0,«Мегафон» стал владельцем 50% акций «Евросети»
2,-1,"RT @fuckkiev: “@EvaKobb: МТС Россия прислала жителям Херсонщины сообщения, в которых обозвала украинцев фашистами? http://t.co/RbSesXlOUZ” …"
3,1,ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехом МТС - Супер 0
4,-1,"@parfenov1960 потому что МТС достало, а пчел ненавижу с детства, как и их мёд!"


In [4]:
test_data.head()

,label,text
0,-1,RT vzglyad: По делу о работе МТС в Узбекистане США предложили заморозить 300 млн
1,0,RT @kevinuyatukox: http://t.co/ljtrjq91v3 #Кредитные карты мегафон банка
2,0,#Оформить кредитную карту в банке мтс http://t.co/vv1B6PMWgH
3,0,#Как перевести деньги с билайна на кредитную карту
4,0,#Начальник отдела кредитного контроля оао мтс усачева н а


### Бейзлайн

**CountVectorizer + логистическая регрессия**

|class|precision|recall|f1-score|support|
|:------:|:------:|:------:|:------:|:------:|
|-1|0.69|0.59|0.64|902|
|0|0.61|0.80|0.69|972|
|1|0.30|0.03|0.06|180|
|avg / total|0.62|0.64|0.61|2054|

* Макросредняя F1 мера -  0.46306421211286786
* Микросредняя F1 мера -  0.6387536514118792

**TF-IDF + логистическая регрессия**

|class|precision|recall|f1-score|support|
|:------:|:------:|:------:|:------:|:------:|
|-1|0.70|0.69|0.70|902|
|0|0.66|0.76|0.71|972|
|1|0.37|0.09|0.15|180|
|avg / total|0.65|0.67|0.65|2054|

* Макросредняя F1 мера -  0.5179519140882505
* Микросредняя F1 мера -  0.6718597857838364

## Задание
По самым характерным словам видно, что в корпусе есть мусор, стоп-слова, разные формы одного слова. Чтобы улучшить результат, попробуйте почистить данные и добавить нормализацию.

### 1. Нормализация и лемматизация

Тексты нормализуются так:

* весь текст приводим к нижнему регистру,
* все слова приводим к их леммам,
* убираем всю пунктуацию,
* убираем обращения (в твиттере для этого используют знак @) и никнеймы (вычищаем их регулярным выражением).

In [5]:
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import string
import re

In [6]:
morph = MorphAnalyzer()
stops = stopwords.words("russian")
punct_extended = string.punctuation + "@«»—…“”"
regex_username = re.compile("[a-zA-Z0-9]+")

In [7]:
def normalize(text):
    """
    функция нормализации
    
    ::takes::
    @text - ненормализованный текст (string)
    
    ::returns::
    нормализованный текст (string)
    """
    text = text.lower()
    tokens = wordpunct_tokenize(text)
    lemmas_raw = [morph.parse(token)[0].normal_form for token in tokens]
    lemmas = [lemma for lemma in lemmas_raw 
              if lemma not in stops
             and lemma not in punct_extended
             and lemma != "rt"
             and not re.search(regex_username, lemma)]
    
    return " ".join(lemmas)

Применяем нормализацию ко всему корпусу.

In [8]:
train_data["normalized"] = train_data["text"].apply(normalize)
test_data["normalized"] = test_data["text"].apply(normalize)

### 2. TF-IDF

Обучаем новую модель на нормализованных данных.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
tfidf = TfidfVectorizer()
tfidf.fit(train_data["normalized"].values)

X_train = tfidf.transform(train_data["normalized"].values)
X_test = tfidf.transform(test_data["normalized"].values)

In [11]:
y_train = train_data["label"].values
y_test = test_data["label"].values

### 3. Ура, машинное обучение!

#### (оригинальное ДЗ) Логистическая регрессия

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [13]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

In [14]:
print(classification_report(y_test, y_pred))
print("Макросредняя F1 мера - ", f1_score(y_test, y_pred, average="macro"))
print("Микросредняя F1 мера - ", f1_score(y_test, y_pred, average="micro"))

             precision    recall  f1-score   support

         -1       0.74      0.57      0.64       902
          0       0.62      0.85      0.72       972
          1       0.48      0.13      0.20       180

avg / total       0.66      0.66      0.64      2054

Макросредняя F1 мера -  0.5208085372311336
Микросредняя F1 мера -  0.6601752677702045


#### Решающее дерево

In [15]:
from sklearn.tree import DecisionTreeClassifier

In [16]:
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)

In [17]:
print(classification_report(y_test, y_pred))
print("Макросредняя F1 мера - ", f1_score(y_test, y_pred, average="macro"))
print("Микросредняя F1 мера - ", f1_score(y_test, y_pred, average="micro"))

             precision    recall  f1-score   support

         -1       0.65      0.51      0.57       902
          0       0.60      0.71      0.65       972
          1       0.21      0.24      0.23       180

avg / total       0.59      0.58      0.58      2054

Макросредняя F1 мера -  0.4820289963638958
Микросредняя F1 мера -  0.5798442064264849


#### Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)

y_pred = forest.predict(X_test)

In [20]:
print(classification_report(y_test, y_pred))
print("Макросредняя F1 мера - ", f1_score(y_test, y_pred, average="macro"))
print("Микросредняя F1 мера - ", f1_score(y_test, y_pred, average="micro"))

             precision    recall  f1-score   support

         -1       0.67      0.53      0.60       902
          0       0.60      0.80      0.69       972
          1       0.54      0.14      0.23       180

avg / total       0.63      0.63      0.61      2054

Макросредняя F1 мера -  0.5039395896930808
Микросредняя F1 мера -  0.626095423563778
